# Physics-Informed Neural Networks - 1D Heat Diffusion

In [7]:
import matplotlib.pyplot as plt
import numpy as np

import os

from utils import save_progress, make_gif, reset_parameters

import IPython

import torch
import torch.nn as nn
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype=torch.float32

### Recap on 1D Heat Diffusion

In heat transfer, one-dimensional heat diffusion describes the distribution of temperature in a material over time due to thermal conduction. This phenomenon is governed by Fourier's law, which states that the heat flux $q$ is proportional to the negative temperature gradient, or:

$q = -k\frac{\partial^2 u}{\partial x^2}$

where $k$ is the thermal conductivity of the material, $u$ is the temperature, and $x$ is the spatial coordinate.

The conservation of energy leads to the heat diffusion equation. If we consider a material with density $\rho$ and specific heat capacity $c_p$, the rate of change of temperature within the material is proportional to the divergence of the heat flux. This gives rise to the one-dimensional heat diffusion equation:

$\frac{\partial u}{\partial t} = \kappa\frac{\partial^2 u}{\partial x^2}$

where $\kappa = \frac{k}{\rho c_p}$ is the thermal diffusivity, a material property that quantifies the rate at which heat diffuses through the material.

In an ideal case with constant thermal properties and no internal heat generation, the above equation describes how temperature evolves spatially and temporally. For example, if a rod is initially at a uniform temperature and then its ends are exposed to different fixed temperatures, the heat diffusion equation predicts how the temperature will vary along the rod and approach a steady-state distribution.

Boundary and initial conditions are essential for solving the heat diffusion equation. Common boundary conditions include fixed temperatures (Dirichlet), fixed heat flux (Neumann), or convective heat transfer at the boundaries.

Over time, the system approaches thermal equilibrium, where the temperature no longer changes with time ($\frac{\partial u}{\partial t} = 0$). In this steady state, the heat diffusion equation reduces to:

$\frac{\partial^2 u}{\partial x^2} = 0$

which indicates that the temperature varies linearly with position if the thermal conductivity is uniform.

### Implementation of the Ground Truth (Analytic Solution)

Here, we consider a 1D heat diffusion case with the following initial condition (initial heat distribution):

$u(x,0) = \sin (\pi x)$

Given the above initial condition, the analytic solution for the 1D heat equation is known to be:

$u(x,t) = e^{-\kappa \pi^2 t}\sin(\pi x)$

In [8]:
# Parameters
L = 1.0        # Length of the physical domain
T = 2.0         # Total time
Nx = 50        # Number of spatial points
Nt = 100      # Number of time steps
alpha = 0.05   # Thermal diffusivity
dx = L / Nx    # Spatial step size
dt = T / Nt    # Time step size

# Discretize the spatial and time domain
t = np.linspace(0, T, Nt+1)  # Time grid
x = np.linspace(0, L, Nx+1)  # Spatial grid

# Initialize temperature array
u = np.zeros((Nt+1, Nx+1))   # u[n, i] corresponds to time step n and position i

# Initial condition: u(0,x) = sin(pi * x)
u[0, :] = np.sin(np.pi * x)

for n in range(1, Nt+1):
    # Analytical solution for the current time step
    u[n, :] = np.exp(-alpha * (np.pi**2) * t[n]) * np.sin(np.pi * x)

# Create a meshgrid for position and time
xx, tt = np.meshgrid(x, t)

Let's plot the result.

In [9]:
def plot_result(
        xx, tt, gt,
        train_x=None, train_t=None, train_u=None,
        colloc_x=None, colloc_t=None,
        prediction=None,
        iter=None
    ):
    # Create the surface plot
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d',computed_zorder=False)
    ax.plot_surface(xx, tt, gt, cmap='viridis', alpha=0.8)
    if not train_u is None:
        ax.scatter(train_x, train_t, train_u, color='red')
    if not colloc_x is None:
        ax.scatter(colloc_x, colloc_t, np.zeros_like(colloc_t), color='green')
    if not prediction is None:
        wire = ax.plot_wireframe(xx, tt, prediction.reshape(xx.shape), color='deepskyblue', linewidth=1, zorder=2)

    # Rotate the plot by setting elevation and azimuth
    ax.view_init(elev=30, azim=60)  # Elevation and azimuth angles

    # Labels and title
    ax.set_xlabel('Position (x)')
    ax.set_ylabel('Time (t)')
    ax.set_zlabel('Temperature (u)')

    iter_string = ''
    if not iter is None:
        iter_string = f' (Iteration={iter+1})'
    ax.set_title('1D Heat Diffusion: Temperature Evolution' + iter_string)

    # Adjust layout to avoid cutting off the z-axis label
    plt.tight_layout()  # Automatically adjust the layout

    return plt

In [ ]:
plot_result(xx, tt, u).show()

### Sample training data

Now to train a model, let's sample training data. Here, we emulate the scenario in which we only know the initial condition at $t=0$ and would like to solve the heat equation for unseen time $t>0$.

In [ ]:
# Sample training data at t=0
N_train = 15
train_x = np.linspace(0, L, N_train)
train_t = np.zeros_like(train_x)
train_u = np.sin(np.pi * train_x)

training_data = np.hstack(( np.expand_dims(train_t,-1), np.expand_dims(train_x,-1),np.expand_dims(train_u,-1)))

# Sample collocation points
N_COLLOCATION_POINTS = 12
xx_colloc, tt_colloc = np.meshgrid(np.linspace(0,L,N_COLLOCATION_POINTS), np.linspace(0,T,N_COLLOCATION_POINTS))
collocation_pts = torch.tensor(np.hstack((tt_colloc.reshape(-1,1), xx_colloc.reshape(-1,1))), dtype=dtype).requires_grad_(True)

plot_result(xx, tt, u, train_x, train_t, train_u, xx_colloc, tt_colloc).show()

### Model Design

Below is a simple MLP implementation as a starter. This will give you a pretty good solution to the diffusion problem, but you are strongly encouraged to play around with other more complex architectures.

In [12]:
class Backbone(nn.Module):
    def __init__(self, dtype=torch.float32):
        super().__init__()

        self.fc1 = nn.Linear(2, 32, dtype=dtype)  # input dim = 2 (t,x)
        self.fc2 = nn.Linear(32, 32, dtype=dtype)  # hidden dims = 32, 32, 32
        self.fc3 = nn.Linear(32, 32, dtype=dtype)  #
        self.out = nn.Linear(32, 1, dtype=dtype)  # output dim = 1 (u)

        self.dtype = dtype

    def forward(self, x):
        x = self.fc1(x)
        x = nn.SiLU()(x)
        x = self.fc2(x)
        x = nn.SiLU()(x)
        x = self.fc3(x)
        x = nn.SiLU()(x)
        return self.out(x)
    
model = Backbone()

### Training

Finally, let's train the model. PINNs are difficult to converge, so you are going to need to tweak with things like loss weights, learning rates, etc. Also, make sure you run a sufficient number of iterations for the model the to converge.

In [ ]:
reset_parameters(model)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4)
files = []

save_dir = 'results/heat1d'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
MAX_ITER = 30000
for iter in range(MAX_ITER):
    optimizer.zero_grad()
    input = torch.tensor(training_data[:,:2], dtype=model.dtype)
    output = torch.tensor(training_data[:,2:], dtype=model.dtype)
    prediction = model(input)
    data_loss = torch.mean((output-prediction)**2)
    
    prediction_colloc = model(collocation_pts)
    # deriv = torch.autograd.grad(prediction_colloc, collocation_pts, torch.ones_like(prediction_colloc), retain_graph=True, create_graph=True)[0]
    deriv = torch.autograd.grad(prediction_colloc, collocation_pts, torch.ones_like(prediction_colloc), retain_graph=True, create_graph=True)[0]
    dt = deriv[:,0]
    dx = deriv[:,1]
    deriv2 = torch.autograd.grad(dx, collocation_pts, torch.ones_like(dx), create_graph=True)[0]
    ddx = deriv2[:,1]
    physics_loss = torch.mean((dt-alpha*ddx)**2)

    loss = data_loss + physics_loss
    loss.backward()
    optimizer.step()

    print(f"{iter+1}/{MAX_ITER} - loss: {loss.detach().numpy():.5e}, physics: {physics_loss.detach().numpy():.5e}", end='\r')
    
    # plot the result as training progresses
    if (iter+1) % 100 == 0: 
        
        prediction = model(torch.tensor(np.hstack((tt.reshape(-1,1),xx.reshape(-1,1))), dtype=model.dtype)).detach()

        plot_result(xx, tt, u, train_x, train_t, train_u, xx_colloc, tt_colloc, prediction, iter)
        files.append(save_progress(save_dir, 'pinn', iter))
        
        if (iter+1) % 5000 == 0: plt.show()
        else: plt.close("all")


In [ ]:
# Animate the training progress
make_gif(files, save_dir + "/pinn.gif")
IPython.display.Image(filename=save_dir + "/pinn.gif") 